In [ ]:
import TraceDisplay
from TraceDisplay import BOKEH_RENDERER
from IPython.display import HTML, display

In [ ]:
image = ! find ${WALK_PATH:-.} -name '*.i.h5'
data_path = image[0]
html_path = '/dev/null'

In [ ]:
image = TraceDisplay.Image()
image.load(data_path)

In [ ]:
bkr = TraceDisplay.BokehRenderer()
bkr.show()

In [ ]:
bkr.render(image)

In [ ]:
# Create a snapshot before calling filter
# bkr.image
display(HTML(bkr.to_html()))
display(image.get_filter()) # TODO: display(image.filter)
display(image['/sched_switch'])

In [ ]:
pid = image['/sched_process_exec'].iloc[0]['common_pid']
display(pid)
image.filter({
    k:f"common_pid == {pid}"
    for k in image
})
# TODO: rewrite update
# image.filter.update({
#     k:f"common_pid == {pid}"
#     for k in image
# })

In [ ]:
# Simulate Button click
bkr.update()
# Create a snapshot after calling filter
# bkr.image
display(HTML(bkr.to_html()))
display(image.get_filter()) # TODO: display(image.filter)
display(image['/sched_switch'])

In [ ]:
# TODO: Append custom categories
#
# image.category.append(
#     label=f'pid == {pid}',
#     color='#FF0000',
#     query = {
#         k:f"common_pid == {pid}"
#         for k in image
#     },
# )
#
# image.category.append(
#     label=f'pid != {pid}',
#     color='#00FF00',
#     query = {
#         k:f"common_pid != {pid}"
#         for k in image
#     },
# )
#
# image.filter.clear()
# bkr.update()
# display(HTML(bkr.to_html()))
#

In [ ]:
# TODO: Create custom shape
#
# image['custom'] = ???
#
# bkr.update()
# display(HTML(bkr.to_html()))
#